In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )
    
def flip_predicate(predicate_id):
    flip_dict = {
        "skos:closeMatch": "skos:relatedMatch",
        "skos:relatedMatch": "skos:closeMatch",
        "skos:narrowMatch" : "skos:broadMatch",
        "skos:broadMatch" : "skos:narrowMatch",
        "skos:exactMatch" : "skos:exactMatch"
    }
    
    return flip_dict[predicate_id]

def compare_and_comment_df(mondo_df, lex_df):
    df = compare_dataframes(mondo_df, lex_df).combined_dataframe
    df['comment'] = df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
    df['comment'] = df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
    return df

def get_unmapped_df(comparison_df):
    mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
    unmapped_df = comparison_df[
        (comparison_df['comment'].str.contains("|".join(mappings)))
    ]

    return unmapped_df

def export_unmatched_exact(unmapped_df, match_type, fn):
    unmapped_exact = unmapped_df[(unmapped_df['comment'] == match_type) & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
    unmapped_exact.to_csv(join(dir_name, fn), sep='\t', index = False)
    return unmapped_exact.head()

def make_msdf(comparison_df, prefix_map, meta):
    combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=prefix_map, metadata=meta)
    return combined_msdf


In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 1min 42s, sys: 1.59 s, total: 1min 44s
Wall time: 1min 46s


In [6]:
mondo_object_prefixes = msdf_mondo.df['object_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
mondo_subject_prefixes = msdf_mondo.df['subject_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
mondo_predicate_ids = msdf_mondo.df['predicate_id'].drop_duplicates()

print(f"mondo_subject_prefixes:\n {mondo_subject_prefixes} \n \
        lex_object_prefixes:\n {mondo_object_prefixes} \n \
        predicate_ids: \n {mondo_predicate_ids}")

mondo_subject_prefixes:
 0    MONDO
Name: subject_id, dtype: object 
         lex_object_prefixes:
 0           SCTID
1            MESH
2            DOID
3        Orphanet
4            NCIT
5            UMLS
11         OMIMPS
51        ICD10CM
172          OMIM
604        MedDRA
657      ICD10WHO
25506      MEDGEN
Name: object_id, dtype: object 
         predicate_ids: 
 0      skos:exactMatch
111    skos:broadMatch
Name: predicate_id, dtype: object


In [7]:
%%time
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
# "ICD10CM", "MONDO"
# msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
# msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))

msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()


CPU times: user 1min 10s, sys: 4.98 s, total: 1min 15s
Wall time: 1min 16s


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
0,MONDO:0000001,disease or disorder,skos:exactMatch,SCTID:64572001,Unspecified,None
1,MONDO:0000001,disease or disorder,skos:exactMatch,MESH:D004194,Unspecified,None
2,MONDO:0000001,disease or disorder,skos:exactMatch,DOID:4,Unspecified,disease
3,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:377788,Unspecified,None
4,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C2991,Unspecified,Disease or Disorder


In [8]:
condition_1 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_2 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_3 = msdf_mondo.df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = msdf_mondo.df['object_id'].str.contains("Orphanet")
condition_5 = msdf_mondo.df['object_id'].str.contains("DOID")

mondo_icd_df = msdf_mondo.df[condition_1 & condition_2]
mondo_omim_df = msdf_mondo.df[condition_1 & condition_3]
mondo_ordo_df = msdf_mondo.df[condition_1 & condition_4]
mondo_doid_df = msdf_mondo.df[condition_1 & condition_5]

mondo_icd_df.head()
mondo_omim_df.head()
mondo_ordo_df.head()
mondo_doid_df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
2,MONDO:0000001,disease or disorder,skos:exactMatch,DOID:4,Unspecified,disease
8,MONDO:0000004,adrenocortical insufficiency,skos:exactMatch,DOID:10493,Unspecified,adrenal cortical hypofunction
14,MONDO:0000009,"inherited bleeding disorder, platelet-type",skos:exactMatch,DOID:2218,Unspecified,blood platelet disease
23,MONDO:0000032,"febrile seizures, familial",skos:exactMatch,DOID:0111297,Unspecified,familial febrile seizures
32,MONDO:0000050,isolated congenital growth hormone deficiency,skos:exactMatch,DOID:0060870,Unspecified,isolated growth hormone deficiency


In [9]:
lex_object_prefixes = msdf_lex.df['object_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
lex_subject_prefixes = msdf_lex.df['subject_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
lex_predicate_ids = msdf_lex.df['predicate_id'].drop_duplicates()

print(f"subject_prefixes:\n {lex_subject_prefixes} \n \
      object_prefixes:\n {lex_object_prefixes} \n \
      predicat_ids:\n {lex_predicate_ids} ")


subject_prefixes:
 0             DOID
130421     ICD10CM
144485    ICD10WHO
147213       MONDO
366217        NCIT
Name: subject_id, dtype: object 
       object_prefixes:
 0        MONDO
7         NCIT
11    ICD10WHO
17        DOID
33     ICD10CM
Name: object_id, dtype: object 
       predicat_ids:
 0       skos:exactMatch
6       skos:broadMatch
14      skos:closeMatch
157    skos:narrowMatch
Name: predicate_id, dtype: object 


### Flipping subject_id and object_id if MONDO not subject_id prefix

**Predicate impact**

- skos:closeMatch <=> skos:relatedMatch
- skos:narrowMatch < => skos:broadMatch

In [10]:
condition_1 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_2 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_3 = msdf_lex.df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = msdf_lex.df['object_id'].str.contains("Orphanet")
condition_5 = msdf_lex.df['object_id'].str.contains("DOID")
condition_mondo_obj = msdf_lex.df['object_id'].str.contains("MONDO")
non_mondo_subjects_df =  pd.DataFrame(msdf_lex.df[(~condition_1 & condition_mondo_obj)])
mondo_subjects_df = pd.DataFrame(msdf_lex.df[(condition_1 & ~condition_mondo_obj)])
print(len(mondo_subjects_df))
non_mondo_subjects_df.head()


7638


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
1,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,angiosarcoma
2,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma
3,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icdo:9120/3
4,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:RegularExpressionReplacement,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma


In [11]:
desired_sequence = ['subject_id', 'subject_label', 'predicate_id', 'object_id',
       'object_label', 'mapping_justification', 'mapping_tool', 'confidence',
       'subject_match_field', 'object_match_field', 'match_string']

new_subjects_df = non_mondo_subjects_df.rename(columns={
    'subject_id':'object_id', 
    'subject_label':'object_label',
    'object_id':'subject_id', 
    'object_label':'subject_label'
    
})


new_subjects_df = new_subjects_df[desired_sequence]
new_subjects_df["predicate_id"] = new_subjects_df["predicate_id"].apply(lambda x: flip_predicate(x))
print(len(new_subjects_df))
new_subjects_df.head()


103220


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
1,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,angiosarcoma
2,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma
3,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icdo:9120/3
4,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:RegularExpressionReplacement,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma


### Combine dfs where subjec_id prefix is MONDO

In [12]:
lex_df = pd.concat([mondo_subjects_df,new_subjects_df], ignore_index=True)
print(len(lex_df))

lex_df.head()

110858


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C25457,Condition,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,condition
1,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C156809,Medical Condition,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,medical condition
2,MONDO:0000004,adrenocortical insufficiency,skos:exactMatch,NCIT:C113211,Hypocortisolemia,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,hypocortisolemia
3,MONDO:0000159,bone marrow failure syndrome,skos:exactMatch,NCIT:C165614,Bone Marrow Failure Syndrome,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,bone marrow failure syndrome
4,MONDO:0000328,hyperphosphatemia,skos:exactMatch,NCIT:C113750,Hyperphosphatemia,semapv:RegularExpressionReplacement,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,hyperphosphatemia


In [13]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = lex_df['subject_id'].str.contains("MONDO")
condition_2 = lex_df['object_id'].str.contains("ICD10CM")
condition_3 = lex_df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = lex_df['object_id'].str.contains("Orphanet")
condition_5 = lex_df['object_id'].str.contains("DOID")


mondo_icd_lex_df = lex_df[(condition_1 & condition_2)]
mondo_omim_lex_df = lex_df[(condition_1 & condition_3)]
mondo_ordo_lex_df = lex_df[(condition_1 & condition_4)]
mondo_doid_lex_df = lex_df[(condition_1 & condition_5)]

mondo_icd_lex_df.head()
mondo_omim_lex_df.head() # NO ROWS
mondo_ordo_lex_df.head() # NO ROWS
mondo_doid_lex_df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
406,MONDO:0004990,breast tumor luminal A or B,skos:narrowMatch,DOID:0060548,luminal breast carcinoma A,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasBroadSynonym,breast tumor luminal
407,MONDO:0004990,breast tumor luminal A or B,skos:narrowMatch,DOID:0060548,luminal breast carcinoma A,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasBroadSynonym,luminal breast cancer
7638,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
7639,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,angiosarcoma
7640,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma


In [14]:
%%time
# comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
# comparison_df = comparison_ms_diff.combined_dataframe
# comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
# comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
# comparison_df.head()

# Comparisons
icd_comparison_df = compare_and_comment_df(mondo_icd_df, mondo_icd_lex_df)
omim_comparison_df = compare_and_comment_df(mondo_omim_df, mondo_omim_lex_df)
ordo_comparison_df = compare_and_comment_df(mondo_ordo_df, mondo_ordo_lex_df)
doid_comparison_df = compare_and_comment_df(mondo_doid_df, mondo_doid_lex_df)


CPU times: user 1min 4s, sys: 593 ms, total: 1min 4s
Wall time: 1min 6s


In [15]:
icd_comparison_df
omim_comparison_df
ordo_comparison_df
doid_comparison_df

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
13471,MONDO:0004761,obsolete urethral diverticulum,skos:exactMatch,DOID:9341,Unspecified,urethral diverticulum,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
572,MONDO:0000425,X-linked disease,skos:exactMatch,DOID:0050735,Unspecified,X-linked monogenic disease,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
71940,MONDO:0100471,vitamin D deficiency,skos:exactMatch,DOID:10574,Unspecified,obsolete vitamin D deficiency,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
66458,MONDO:0021668,obsolete disorder involving pain,skos:exactMatch,DOID:0060164,Unspecified,pain disorder,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
6179,MONDO:0002324,obsolete enamel erosion,skos:exactMatch,DOID:2497,Unspecified,enamel erosion,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7938,MONDO:0007872,LADD syndrome,skos:exactMatch,DOID:0050331,semapv:LexicalMatching,LADD syndrome,COMMON_TO_BOTH,oaklib,0.849779,rdfs:label,rdfs:label,ladd syndrome
279,MONDO:0000262,otomycosis,skos:exactMatch,DOID:0050147,Unspecified,otomycosis,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN
7780,MONDO:0000262,otomycosis,skos:exactMatch,DOID:0050147,semapv:LexicalMatching,otomycosis,COMMON_TO_BOTH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d059249
7781,MONDO:0000262,otomycosis,skos:exactMatch,DOID:0050147,semapv:LexicalMatching,otomycosis,COMMON_TO_BOTH,oaklib,0.849779,rdfs:label,rdfs:label,otomycosis


In [16]:
icd_comparison_df['comment'].drop_duplicates()

60989     MONDO_MAPPINGS
105217          LEXMATCH
2483      COMMON_TO_BOTH
Name: comment, dtype: object

### Split into unmapped dataframes

In [17]:
unmapped_icd_df = get_unmapped_df(icd_comparison_df)
unmapped_omim_df = get_unmapped_df(omim_comparison_df)
unmapped_ordo_df = get_unmapped_df(ordo_comparison_df)
unmapped_doid_df = get_unmapped_df(doid_comparison_df)

unmapped_icd_df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
60989,MONDO:0019136,Zygomycosis,skos:broadMatch,ICD10CM:B46.8,Unspecified,Other zygomycoses,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
50419,MONDO:0015153,obsolete autosomal monosomy,skos:broadMatch,ICD10CM:Q93.7,Unspecified,Deletions with other complex rearrangements,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
18103,MONDO:0006143,cervical squamous cell carcinoma,skos:broadMatch,ICD10CM:C53.8,Unspecified,Malignant neoplasm of overlapping sites of cer...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
63620,MONDO:0020054,obsolete partial autosomal monosomy,skos:broadMatch,ICD10CM:Q93.5,Unspecified,Other deletions of part of a chromosome,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
50347,MONDO:0015131,combined immunodeficiency,skos:broadMatch,ICD10CM:D81.2,Unspecified,Severe combined immunodeficiency [SCID] with l...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [19]:
# Let reviewers check if this makes sense or no.
export_unmatched_exact(unmapped_icd_df, "LEXMATCH", "unmapped_icd_lex.tsv")
export_unmatched_exact(unmapped_omim_df, "LEXMATCH", "unmapped_omim_lex.tsv")
export_unmatched_exact(unmapped_ordo_df, "LEXMATCH", "unmapped_ordo_lex.tsv")
export_unmatched_exact(unmapped_doid_df, "LEXMATCH", "unmapped_doid_lex.tsv")


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
14227,MONDO:0009760,Norman-Roberts syndrome,skos:exactMatch,DOID:0060278,semapv:LexicalMatching,pontocerebellar hypoplasia type 9,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:q04.3
103562,MONDO:0014420,short stature due to primary acid-labile subun...,skos:exactMatch,DOID:9521,semapv:LexicalMatching,Laron syndrome,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:e34.3
60999,MONDO:0015149,pure hereditary spastic paraplegia,skos:exactMatch,DOID:0110763,semapv:LexicalMatching,hereditary spastic paraplegia 10,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:g11.4
17282,MONDO:0013090,chromosome 19q13.11 deletion syndrome,skos:exactMatch,DOID:0060407,semapv:LexicalMatching,chromosome 18q deletion syndrome,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:q93.5
32721,MONDO:0017666,diffuse palmoplantar keratoderma,skos:exactMatch,DOID:0070134,semapv:LexicalMatching,autosomal recessive cutis laxa type IIA,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:q82.8


In [21]:
# Inspect why these are missing from SSSOM mappings
export_unmatched_exact(unmapped_icd_df, "MONDO_MAPPINGS", "unmapped_icd_mondo.tsv")
export_unmatched_exact(unmapped_omim_df, "MONDO_MAPPINGS", "unmapped_omim_mondo.tsv")
export_unmatched_exact(unmapped_ordo_df, "MONDO_MAPPINGS", "unmapped_ordo_mondo.tsv")
export_unmatched_exact(unmapped_doid_df, "MONDO_MAPPINGS", "unmapped_doid_mondo.tsv")

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
13471,MONDO:0004761,obsolete urethral diverticulum,skos:exactMatch,DOID:9341,Unspecified,urethral diverticulum,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
572,MONDO:0000425,X-linked disease,skos:exactMatch,DOID:0050735,Unspecified,X-linked monogenic disease,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
71940,MONDO:0100471,vitamin D deficiency,skos:exactMatch,DOID:10574,Unspecified,obsolete vitamin D deficiency,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
66458,MONDO:0021668,obsolete disorder involving pain,skos:exactMatch,DOID:0060164,Unspecified,pain disorder,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
6179,MONDO:0002324,obsolete enamel erosion,skos:exactMatch,DOID:2497,Unspecified,enamel erosion,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # Add distances
# # Add Levenshtein distance [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# # Add Jaccard Index [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# # Add Monge-Elkan Distance [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
# unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
# unmapped_mondo_exact.head()


In [30]:
combined_msdf_icd = make_msdf(icd_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_omim = make_msdf(omim_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_ordo = make_msdf(ordo_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_doid = make_msdf(doid_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)

combined_msdf_icd.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
60989,MONDO:0019136,Zygomycosis,skos:broadMatch,ICD10CM:B46.8,Unspecified,Other zygomycoses,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
50419,MONDO:0015153,obsolete autosomal monosomy,skos:broadMatch,ICD10CM:Q93.7,Unspecified,Deletions with other complex rearrangements,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
18103,MONDO:0006143,cervical squamous cell carcinoma,skos:broadMatch,ICD10CM:C53.8,Unspecified,Malignant neoplasm of overlapping sites of cer...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
63620,MONDO:0020054,obsolete partial autosomal monosomy,skos:broadMatch,ICD10CM:Q93.5,Unspecified,Other deletions of part of a chromosome,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
50347,MONDO:0015131,combined immunodeficiency,skos:broadMatch,ICD10CM:D81.2,Unspecified,Severe combined immunodeficiency [SCID] with l...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [ ]:
# %%time
# df_dict = split_dataframe(combined_msdf)

In [ ]:
# mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
# mondo_icd_list

In [ ]:
# for match in mondo_icd_list:
#     fn = match + ".tsv"
#     df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [ ]:
# df_dict['mondo_exactmatch_icd10cm'].df